# Project Euler

> Lisez Euler, lisez Euler, c'est notre maître à tous

-- Pierre-Simon de Laplace

## Prelude

In [36]:
#type: ignore
from math import gcd, sqrt, ceil, log, floor
from itertools import count, takewhile, dropwhile, islice, chain, permutations, product
from functools import reduce, lru_cache
from datetime import datetime, timedelta
from collections import defaultdict, Counter
from operator import mul
from typing import List, Tuple
from fractions import Fraction as F

phi = (sqrt(5) + 1) / 2


def prod(u): 
    """Product. Older Pythons don't have this."""
    return reduce(mul, u, 1)


def lcm(*u):
    """Least common multiple. Older Pythons don't have this."""
    def lcm_2(a, b):
        return a * b // gcd(a, b)

    
    return reduce(lcm_2, u, 1)


def digits(n):
    """Little-endian list of the digits of `n` in base `10`."""
    return [int(u) for u in str(n)][::-1]


def digits_big(n):
    """Big-endian list of the digits of `n` in base `10`."""
    return [int(u) for u in str(n)]


def undigits(d):
    """Assemble a little-endian list of digits into the number it represents."""
    return sum(10**e * n for e, n in enumerate(d))


def undigits_big(d):
    """Assemble a big-endian list of digits into the number it represents."""
    return sum(10**e * n for e, n in enumerate(d[::-1]))


def primes():
    """Infinite generator that returns primes."""
    ps = defaultdict(list)
    for i in count(2):
        if i in ps:
            for n in ps[i]:
                ps[i + (n if n == 2 else 2*n)].append(n)
            del ps[i]
        else:
            yield i
            ps[i**2].append(i)

            
def factorize(n: int):
    """
    Factorize a number. Return a Counter `(p, e)`
    where `p` is the prime, `e` is the exponent.
    """
    fs = Counter()
    gen = primes()
    while n > 1:
        p = next(gen)
        while n % p == 0:
            n //= p
            fs[p] += 1

    return fs


def divisors(n: int):
    fs = factorize(n)
    def __div_helper(k: Counter[int]):
        if not k:
            return [1]
        else:
            (p, e) = k.popitem()
            R = __div_helper(k)
            return [p ** d * r for d in range(1 + e) for r in R]
        
    return sorted(__div_helper(fs))

## [Problem 1 - Multiples of 3 or 5](https://projecteuler.net/problem=1)

There are $333$ multiples of $3$ less than $1000$, $199$ multiples of $5$ less than $1000$, and $66$ multiples of $15$ less than $1000$. Let $S$ be the sum we are seeking. Then:

$$
S = \sum_{i=1}^{333} 3i + \sum_{i=1}^{199} 5i - \sum_{i=1}^{66} 15i
$$

From which:

$$
S = \frac{3 \times 333 \times 334}{2} + \frac{5 \times 199 \times 200}{2} - \frac{15 \times 66 \times 67}{2} = 233168
$$

We may verify this is correct solving the problem the naive way:

In [37]:
A = sum(i for i in range(1000) if i % 3 == 0 or i % 5 == 0)

assert A == 233_168

## [Problem 2 - Even Fibonacci numbers](https://projecteuler.net/problem=2)

The problem can be solved with pen and paper! Yay for $O(0)$ solutions!

We are looking for the sum of even Fibonacci number less than $N = 4 \times 10^6$; let that sum be $A$.

Let $F_n$ be the $n$-th Fibonacci number, and let $L$ be the greatest integer such that $F_L < N$.

Because $gcd(F_n, F_m) = F_{gcd(n, m)}$, $F_n$ is even if and only if $3 | n$. Therefore:

$$
A =
\sum_{0 \le j \le L} \left[ F_j < N\right] F_j =
\sum_{0 \le 3j \le L} F_{3j}
$$

Duplicating $A$, we obtain:

$$
2A = 2 \sum_{0 \le 3j \le L} F_{3j} = F_0 + F_0 + F_3 + F_3 + \dots
$$

Because $F_1 = 1$, we may write:

$$
\begin{array}{rcl}
2A + 1 & = & F_0 + (F_0 + F_1) + F_3 + F_3 + \dots \\
& = & F_0 + (F_2 + F_3) + F_3 + F_6 + F_6 + \dots \\
& = & F_0 + (F_3 + F_4) + F_6 + F_6 + \dots
\end{array}
$$

Which telescopically reduces to:

$$
2A + 1 = F_{L + 1}
$$

This suggests a way to directly compute the answer. We know that:

$$
F_n = \left \lfloor \frac{\varphi^n}{\sqrt{5}} + \frac{1}{2} \right \rfloor
$$

Which can be inverted as:

$$
n(F) = \left \lceil \frac{1}{\log{\varphi}} \log \left( F \sqrt{5} - \frac{1}{2} \right) \right \rceil
$$

Plugging in $N = 4 \times 10^6$:

In [38]:
N = 4_000_000
ceil(log(sqrt(5) * N - 1/2) / log(phi))

34

We deduce that $T = 34$. Hence:

In [39]:
L = 34
(floor(phi ** (L + 1) / sqrt(5) + 1/2) - 1) // 2

4613732

Yielding the final answer. We may check that naively computing the answer gives the same result.

In [40]:
def fibonacci():
    prev, curr = 0, 1
    while True:
        yield prev
        prev, curr = curr, prev + curr


N = 4_000_000
A = sum(i for i in takewhile(lambda x: x < N, fibonacci()) if i % 2 == 0)


assert A == 4_613_732

## [Problem 3 - Largest prime factor](https://projecteuler.net/problem=3)

In [41]:
N = 600_851_475_143
A = max(p for p in factorize(N))

assert A == 6_857

## [Problem 4 - Largest palindrome product](https://projecteuler.net/problem=4)

In [42]:
R = (100, 1000)

def is_palindrome(n):
    return str(n) == str(n)[::-1]

A  = max(
    x * y
    for x in range(*R)
    for y in range(*R)
    if is_palindrome(x * y)
)


assert A == 906_609

## [Problem 5 - Smallest multiple](https://projecteuler.net/problem=5)

In [43]:
N = 20
A = lcm(*range(1, 1 + N))

assert A == 232_792_560

## [Problem 6 - Sum square difference](https://projecteuler.net/problem=6)

$$
A =
\left( \sum_{1 \le i \le n} i \right)^2 - \sum_{1 \le i \le n} i^2 =
\frac{n^2 (n + 1)^2}{4} - \frac{n(n+1)(2n+1)}{6} =
\frac{3n^4 + 2n^3 - 3n^2 - 2n}{12}
$$

Plugging in $n=100$:

In [44]:
N = 100
(3 * N ** 4 + 2 * N ** 3 - 3 * N ** 2 - 2 * N) // 12

25164150

We may verify that running the naive algorithm yields the same result:

In [45]:
N = 100
A = sum(range(1 + N)) ** 2 - sum(x**2 for x in range(1 + N))


assert A == 25_164_150

## [Problem 7 - 10001st prime](https://projecteuler.net/problem=7)

In [46]:
N=10_001
A = next(islice(primes(), N - 1, N))

assert A == 104_743

## [Problem 8 - Largest product in a series](https://projecteuler.net/problem=8)

In [47]:
with open("input/8.txt") as f:
    S = [i for l in [[int(x) for x in line.strip()] for line in f] for i in l]

N = 13
A = max(prod(S[i : i + N]) for i in range(len(S) - N))


assert A == 23_514_624_000

## [Problem 9 - Special Pythagorean triple](https://projecteuler.net/problem=7)

In [48]:
N = 1000
A = next(
    a * b * c
    for a in range(1, N)
    for b in range(N - a)
    for c in (N - a - b,)
    if a**2 + b**2 == c**2
)


assert A == 31_875_000

## [Problem 10 - Summation of primes](https://projecteuler.net/problem=10)

In [49]:
N = 2_000_000
A = sum(takewhile(lambda x: x < N, primes()))

assert A == 142_913_828_922

## [Problem 11 - Largest product in a grid](https://projecteuler.net/problem=11)

In [50]:
with open("input/11.txt") as f:
    M = [[int(x) for x in line.strip().split()] for line in f]

horizontals = [x for x in M]

verticals = [[x[i] for x in M] for i in range(len(M))]

primary_diag = [
    [M[i][j] for i in range(len(M)) for j in range(len(M)) if i - j == d]
    for d in range(-len(M), len(M))
]

secondary_diag = [
    [M[i][j] for i in range(len(M)) for j in range(len(M)) if i + j == d]
    for d in range(2 * len(M) - 1)
]

N = 4
A = 0
for l in chain(horizontals, verticals, primary_diag, secondary_diag):
    top = max((prod(l[i : i + N]) for i in range(1 + len(l) - N)), default=0)
    A = max(top, A)

assert A == 70_600_674

## [Problem 12 - Highly divisible triangular number](https://projecteuler.net/problem=12)

In [51]:
def triangular(n):
    return n * (n + 1) // 2

def factors(n):
    return prod(1 + e for e in factorize(n).values())

N=500
A = next(dropwhile(
    lambda u: factors(u) <= N,
    (triangular(x) for x in count())
))


assert A == 76_576_500

## [Problem 13 - Large sum](https://projecteuler.net/problem=13)

In [52]:
N=10
with open("input/13.txt") as f:
    L = [int(l.strip()) for l in f]
    
A = int(str(sum(L))[:N])

assert A == 5_537_376_230

## [Problem 14 - Longest collatz sequence](https://projecteuler.net/problem=14)

In [53]:
@lru_cache(maxsize=None)
def collatz_length(n):
    if n == 1:
        return 0
    elif n % 2 == 1:
        return 1 + collatz_length(3 * n + 1)
    else:
        return 1 + collatz_length(n // 2)


N = 1_000_000
A = max(range(1, N), key=collatz_length)


assert A == 837_799

## [Problem 15 - Lattice Paths](https://projecteuler.net/problem=15)

In [54]:
N = 20
A = prod(range(N + 1, 2 * N + 1)) // prod(range(1, N + 1))

assert A == 137846528820

## [Problem 16 - Power digit sum](https://projecteuler.net/problem=16)

In [55]:
N = 2 ** 1000
A = sum(digits(N))

assert A == 1366

## [Problem 17 - Number letter counts](https://projecteuler.net/problem=17)

In [56]:
def number_to_british_english(n):
    MAX_SCALE = 10 ** 21
    if n >= MAX_SCALE:
        raise

    names_special = {
        10: 'ten',
        11: 'eleven',
        12: 'twelve',
        13: 'thirteen',
        14: 'fourteen',
        15: 'fifteen',
        16: 'sixteen',
        17: 'seventeen',
        18: 'eighteen',
        19: 'nineteen'
    }
    names_u = {
        1: 'one',
        2: 'two',
        3: 'three',
        4: 'four',
        5: 'five',
        6: 'six',
        7: 'seven',
        8: 'eight',
        9: 'nine'
    }
    names_da = {
        2: 'twenty',
        3: 'thirty',
        4: 'forty',
        5: 'fifty',
        6: 'sixty',
        7: 'seventy',
        8: 'eighty',
        9: 'ninety'
    }
    names_scale = {
        1: 'thousand',
        2: 'million',
        3: 'billion',
        4: 'trillion',
        5: 'quadrillion',
        6: 'quintillion'
    }
    dl = digits(n)[::-1]
    d = ([0] * (3 - (len(dl) % 3 or 3)) + dl)[::-1]
    s = [list(d[3 * k : 3 * k + 3]) for k in range(ceil(len(d) / 3))]
    
    if not s:
        return 'zero'

    out = ''
    for k, (u, da, h) in enumerate(s):
        prefix = ' ' + names_scale[k] + ' ' if k else ''
        hds = names_u[h] + ' hundred' if h else ''
        conn = ' and ' if h and (da or u) else ''
        if da == 0 and u == 0:
            base = ''
        elif da * 10 + u in names_special:
            base = names_special[da * 10 + u]
        elif da == 0:
            base = names_u[u]
        elif u == 0:
            base = names_da[da]
        else:
            base = f'{names_da[da]}-{names_u[u]}'

        out = hds + conn + base + prefix + out
        

    return out

N = 1000
A = sum(sum(1 for c in number_to_british_english(n) if c.isalpha()) for n in range(1, N+1))

assert A == 21124

## [Problem 18 - Maximum path sum I](https://projecteuler.net/problem=18)

In [57]:
V = []
with open('input/14.txt') as f:
    for l in f:
        V.append([int(x) for x in l.strip().split()])

def gen_max_path(A):
    @lru_cache(maxsize=None)
    def __max_path(row, col):
        if row == 0:
            return A[0][0]
        elif col == 0:
            return A[row][col] + __max_path(row-1, col)
        elif col == row:
            return A[row][col] + __max_path(row-1, col-1)
        else:
            return A[row][col] + max(__max_path(row-1, col-1), __max_path(row-1, col))
        
    return __max_path

max_path = gen_max_path(V)
A = max(max_path(len(V) - 1, i) for i in range(len(V)))

assert A == 1074

## [Problem 19 - Counting Sundays](https://projecteuler.net/problem=19)

In [58]:
def date_range():
    d = datetime(1901, 1, 1)
    while d < datetime(2001, 1, 1):
        yield d
        d += timedelta(days = 1)
        
A = sum(1 for x in date_range() if x.weekday() == 6 and x.day == 1)

assert A == 171

## [Problem 20 - Factorial digit sum](https://projecteuler.net/problem=20)

In [59]:
N = 100
A = sum(digits(prod(range(1, 1 + N))))

assert A == 648

## [Problem 21 - Amicable numbers](https://projecteuler.net/problem=21)

In [60]:
N = 10000
sum_divisors = {i: sum(divisors(i)) - i for i in range(1, N)}

A = sum(i for i, j in sum_divisors.items() if i != j and sum_divisors.get(j) == i)

assert A == 31626

## [Problem 22 - Names scores](https://projecteuler.net/problem=22)

In [61]:
with open('input/22.txt') as f:
    N = f.read().replace('"', '').split(',')
    
def val(s):
    return sum(1 + ord(c) - ord('a') for c in s.lower())

A = sum(n * val(s) for n, s in enumerate(sorted(N), 1))

assert A == 871198282

## [Problem 23 - Non-abundant sums](https://projecteuler.net/problem=23)

In [62]:
N = 28123
abundant = [j for j in range(1, N + 1) if sum(divisors(j)) - j > j]
A = sum(set(range(1, N + 1)) - set(x + y for x in abundant for y in abundant))

assert A == 4179871

## [Problem 24 - Lexicographic permutations](https://projecteuler.net/problem=24)

The permutation may be found by writing $10^6 - 1$ in factorial base:

$$
\begin{array}{rcl}
999999 & = & 2 \times 362880 + 274239 \\
274239 & = & 6 \times 40320 + 32319 \\
32319 & = & 6 \times 5040 + 2079 \\
2079 & = & 2 \times 720 + 639 \\
639 & = & 5 \times 120 + 39 \\
39 & = & 1 \times 24 + 15 \\
15 & = & 2 \times 6 + 3 \\
3 & = & 1 \times 2 + 1 \\
1 & = & 1 \times 1 + 0 \\
0 & = & 0 \times 0 + 0 \\
\end{array}
$$

..and taking the $k$-th element in order for each digit in the factorial base representation of $10^6 - 1$:
$$
\begin{array}{rcl}
S = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] & \implies & S[2] = 2 \\
S = [0, 1,    3, 4, 5, 6, 7, 8, 9] & \implies & S[6] = 7 \\
S = [0, 1,    3, 4, 5, 6,    8, 9] & \implies & S[6] = 8 \\
S = [0, 1,    3, 4, 5, 6,       9] & \implies & S[2] = 3 \\
S = [0, 1,       4, 5, 6,       9] & \implies & S[5] = 9 \\
S = [0, 1,       4, 5, 6         ] & \implies & S[1] = 1 \\
S = [0,          4, 5, 6         ] & \implies & S[2] = 5 \\
S = [0,          4,    6         ] & \implies & S[1] = 4 \\
S = [0,                6         ] & \implies & S[1] = 6 \\
S = [0                           ] & \implies & S[0] = 0 \\
\end{array}
$$

Yielding: $[2, 7, 8, 3, 9, 1, 5, 4, 6, 0]$.

This may be verified comparing the permutation thus obtained with a naive approach:

In [63]:
N = 1_000_000
A = next(islice(permutations(range(10)), N-1, N))

assert A == (2, 7, 8, 3, 9, 1, 5, 4, 6, 0)

## [Problem 25 - 1000-digit Fibonacci number](https://projecteuler.net/problem=25)

In [64]:
N = 10**999

def fibonacci():
    prev, curr = 0, 1
    while True:
        yield prev
        prev, curr = curr, prev + curr

A, _ = next(dropwhile(lambda u: u[1] < N, enumerate(fibonacci())))

assert A == 4782

## [Problem 26 - Reciprocal cycles](https://projecteuler.net/problem=26)

In [65]:
N = 1_000

def cycle_length(n):
    s = {1}
    v = 1
    while True:
        v *= 10
        v %= n
        if v == 0:
            break
        if v in s:
            break
        s |= {v}
        
    return len(s)

A = max(range(1, N+1), key=cycle_length)

assert A == 983

## [Problem 27 - Quadratic primes](https://projecteuler.net/problem=27)

In [66]:
R = (-999, 1000)

def quad(a, b):
    return lambda n: n ** 2 + a * n + b

pf, mpf, pg = set(), -1, primes()

def npr(f):
    global pf, mpf, pg
    for n in count():
        y = f(n)
        while mpf < y:
            mpf = next(pg)
            pf.add(mpf)
        if y not in pf:
            return n

A = prod(max(((a, b) for a in range(*R) for b in range(*R)), 
             key=lambda u: npr(quad(*u))))

assert A == -59231

## [Problem 28 - Number spiral diagonals](https://projecteuler.net/problem=28)

Let $S(n)$ be the sum of the diagonals for an $n \times n$ board.

If we ignore the central $1$, the $k$-th spiral from the center contains the following four numbers:

$$
    (2k+1)^2, (2k+1)^2 - 2k, (2k+1)^2 - 4k, (2k+1)^2 - 6k
$$

Because an $n \times n$ grid contains exactly $(n-1)/2$ spirals (plus the central $1$), the sum is simply:

$$
    S(n) - 1 = \sum_{i=1}^{h} 4(2k+1)^2 - 12k
$$

Where $h = (n-1)/2$. By trivial algebraic manipulation:

$$
    S(n) - 1 = 4 \sum_{i=1}^{h} (2k+1)^2 - 3k 
    = 4 \sum_{i=1}^{h} 4k^2 + k + 1 
    = \frac{16h(h+1)(2h+1)}{6} + 2h(h+1) + 4h
$$

Which simplifies to:

$$
    S(n) - 1 = \frac{16h^3 + 30h^2 + 26h}{3}
$$

In our case, $h=500$, which yields the final answer:

In [67]:
H = 500
A = 1 + (16 * H**3 + 30 * H**2 + 26*H) // 3

assert A == 669171001

## [Problem 29 - Distinct powers](https://projecteuler.net/problem=29)

In [68]:
R = (2, 101)
A = len(set(a ** b for a in range(*R) for b in range(*R)))

assert A == 9183

## [Problem 30 - Digit fifth powers](https://projecteuler.net/problem=30)

Let $f(n)$ be the sum of the fifth powers of the digits of $n$. Because $n$ has $1 + \left \lfloor \log_{10}n \right \rfloor$ digits, it holds:

$$
f(n) \le 9^5 + 9^5 \log_{10}n
$$

Because $f(n)$ grows less than $n$, the set of integers equal to the sum of the fifth powers of their digits is bounded above, for example by $9^6$:

$$
9^6 \gt 9^5 + 6 \times 9^5 \gt 9^5 + 9^5 \log_{10} 9^5 \gt f(9^6)
$$

In [69]:
A = sum(x for x in range(10, 9**6) if x == sum(i**5 for i in digits(x)))

assert A == 443839

## [Problem 31 - Coin sums](https://projecteuler.net/problem=31)

In [70]:
coins = (1, 2, 5, 10, 20, 50, 100, 200)
N = 200

@lru_cache(maxsize=None)
def coin_sums(n):
    if n < 0:
        return set()
    if n == 0:
        return {tuple(0 for _ in coins)}
    R = set()
    for i in range(len(coins)):
        D = coin_sums(n - coins[i])
        for t in D:
            R.add(tuple(
                t[j] if j != i else t[i] + 1 for j in range(len(coins))
            ))
    return R

A = len(coin_sums(N))

assert A == 73682

## [Problem 32 - Pandigital products](https://projecteuler.net/problem=32)

In [71]:
S = set()
for i in product(*('abc' for _ in range(9))):
    p = defaultdict(set)
    for x, y in zip(i, range(1, 10)):
        p[x].add(y)
    for a, b, c in product(*(permutations(p[x]) for x in 'abc')):
        if undigits(a) * undigits(b) == undigits(c):
            S.add(undigits(c))

A = sum(S)
assert A == 45228

## [Problem 33 - Digit cancelling fractions](https://projecteuler.net/problem=33)

In [72]:
R = (10, 100)

def is_curious(x, y):
    xd = digits(x)
    yd = digits(y)

    if x % 10 == 0 and y % 10 == 0:
        return False

    if not any(j in yd for j in xd):
        return False

    xv = undigits(d for d in xd if d not in yd)
    yv = undigits(d for d in yd if d not in xd)
    return x * yv == y * xv and xv != 0 and yv != 0

S = set((a, b) for a in range(*R) for b in range(*R) if is_curious(a, b))
A = prod(F(a, b) for a, b in S if a < b)

assert A == F(1, 100)

## [Problem 34 - Digit factorials](https://projecteuler.net/problem=34)

Let $f(n)$ be the sum of the factorials of the digits of $n$. Because $n$ has $1 + \left \lfloor \log_{10}n \right \rfloor$ digits, it holds:

$$
f(n) \le 9! + 9! \log_{10}n
$$

Because $f(n)$ grows less than $n$, the set of integers equal to the sum of the facrorials of their digits is bounded above, for example by $10!$:

$$
10! = 9! + 9! \times 9 \gt 9! + 9! \log_{10} 10! \gt f(10!)
$$

In [73]:
factorial = lambda u: prod(range(1, u+1))
A = sum(x for x in range(10, factorial(10)) if x == sum(factorial(i) for i in digits(x)))

assert A == 40730

## [Problem 35 - Circular primes](https://projecteuler.net/problem=35)

In [74]:
N = 1_000_000
P = {*takewhile(lambda u: u < N, primes())}

def rotations(n):
    d = digits(n)
    return {undigits(d[i:] + d[:i]) for i in range(len(d))}

A = sum(1 for x in P if all(i in P for i in rotations(x)))

assert A == 55

## [Problem 36 - Double-base palindromes](https://projecteuler.net/problem=36)

In [75]:
N = 1_000_000
p = {x for x in range(N) if str(x) == str(x)[::-1]}
q = {x for x in p if str(bin(x))[2:] == str(bin(x))[:1:-1]}
A = sum(q)

assert A == 872187

## [Problem 37 - Truncatable primes](https://projecteuler.net/problem=37)

In [76]:
pf, mpf, pg = set(), -1, primes()

def is_prime(f):
    global pf, mpf, pg
    while f > mpf:
        p = next(pg)
        mpf = p
        pf.add(p)
    return f in pf

def is_truncatable(n):
    d = digits(n)
    for i in range(len(d)):
        if not is_prime(undigits(d[i:])):
            return False
    for i in range(1, len(d) + 1):
        if not is_prime(undigits(d[:i])):
            return False
    return True

def list_truncatable():
    r = 11
    for n in count(11):
        if is_truncatable(n):
            yield n
            r -= 1
            if not r:
                break
                
A = sum(list_truncatable())

assert A == 748317

## [Problem 38 - Pandigital multiples](https://projecteuler.net/problem=38)

In [77]:
def is_pandigital(n):
    return n < 10**10 and set(digits(n)) == set(range(1, 10))


def max_pandigital():
    m = -1
    for n in range(10000):
        for j in range(2, 1 + 10 // len(digits(n))):
            R = range(1, j+1)
            x = int(''.join((str(n * u) for u in R)))
            if is_pandigital(x):
                m = max(m, x)
    return m
            
A = max_pandigital()

assert A == 932718654

## [Problem 39 - Integer right triangles](https://projecteuler.net/problem=39)

In [78]:
d = defaultdict(int)
for i in range(1, 1001):
    for j in range(i + 1, 1001 - i):
        k = (i ** 2 + j ** 2) ** .5
        if k == int(k):
            d[i + j + int(k)] += 1
            
A = max(
    ((u, v) for u, v in d.items() if u <= 1000), 
    key = lambda u: u[1]
)[0]

assert A == 840

## [Problem 40 - Champerowne's constant](https://projecteuler.net/problem=40)

In [79]:
def champ():
    for n in count(1):
        for d in reversed(digits(n)):
            yield d
            
def take_some(it, l, start=0):
    return {
        n + start: x 
        for n, x in enumerate(islice(it, 0, max(l)))
        if n + start in l
    }

A = prod(take_some(champ(), {10**u for u in range(7)}, start=1).values())

assert A == 210

## [Problem 41 - Pandigital prime](https://projecteuler.net/problem=41)

It is sufficient to observe that nine- and eight-digit pandigital numbers can't be primes: the sum of their digits is a multiple of $3$.

In [80]:
P = list(takewhile(lambda u: u < 10**5, primes()))

def is_prime(n):
    return n >= 2 and all((n % p != 0) for p in takewhile(lambda u: u * u <= n, P))
        
A = max(filter(
    is_prime, 
    (undigits(j) for j in permutations(range(1, 8)))
))

assert A == 7652413

## [Problem 42 - Coded triangle numbers](https://projecteuler.net/problem=42)

If $n$ is a triangle, then there must exist a $k$ such that:

$$
n = \frac{k(k+1)}{2}
$$

Hence:

$$
k^2 + k - 2n = 0 
$$

Which is true for:

$$
k = \frac{-1 \pm \sqrt{1 + 8n}}{2}
$$

In [81]:
with open('input/42.txt') as f:
    N = f.read().replace('"', '').split(',')
    
def val(s):
    return sum(1 + ord(c) - ord('a') for c in s.lower())

def is_triangle(n):
    k = ((1 + 8 * n) ** .5 - 1) / 2
    return k == int(k)

A = sum(1 for u in N if is_triangle(val(u)))

assert A == 162

## [Problem 43 - Sub-string divisibility](https://projecteuler.net/problem=43)

In [82]:
def is_divisible(d):
    def num(a, b, c):
        return 100 * d[a] + 10 * d[b] + d[c]
    return (
        num(1, 2, 3) % 2 == 0 and
        num(2, 3, 4) % 3 == 0 and
        num(3, 4, 5) % 5 == 0 and
        num(4, 5, 6) % 7 == 0 and
        num(5, 6, 7) % 11 == 0 and
        num(6, 7, 8) % 13 == 0 and
        num(7, 8, 9) % 17 == 0
    )

A = sum(
    sum(10**e * n for e, n in enumerate(d[::-1]))
    for d in permutations(range(10)) 
    if is_divisible(d)
)

assert A == 16695334890

## [Problem 44 - Pentagon numbers](https://projecteuler.net/problem=44)

In [83]:
N = {n * (3 * n - 1) // 2 for n in range(1, 10 ** 4)}


def find_minimum_difference(S):
    for i in S:
        for j in S:
            if i == j:
                continue
            if i + j in S and abs(i - j) in S:
                return abs(i - j)


A = find_minimum_difference(N)

assert A == 5482660

## [Problem 45 - Triangular, pentagonal and hexagonal](https://projecteuler.net/problem=45)

In [84]:
def triangular():
    for n in count(1):
        yield n * (n+1) // 2
        
def pentagonal():
    for n in count(1):
        yield n * (3 * n - 1) // 2
        
def hexagonal():
    for n in count(1):
        yield n * (2 * n - 1)
        
def find_eq(low, *gen):
    state = [(u, next(u)) for u in gen]
    while True:
        n = state[0][1]
        if all(u == n for _, u in state) and n > low:
            break
        m, _ = min(enumerate(state), key=lambda u: u[1][1])
        g, _ = state[m]
        state[m] = g, next(g)
    return n
    
    
A = find_eq(40755, triangular(), pentagonal(), hexagonal())

assert A == 1533776805

## [Problem 46 - Goldbach's other conjecture](https://projecteuler.net/problem=46)

In [85]:
pf, mpf, pg = set(), -1, primes()

def is_prime(f):
    global pf, mpf, pg
    while f > mpf:
        p = next(pg)
        mpf = p
        pf.add(p)
    return f in pf


def is_square(n):
    if n < 0:
        return False
    return sqrt(n) == int(sqrt(n))


def min_counterexample():
    for i in count(9):
        if i % 2 == 0:
            continue
        if is_prime(i):
            continue
        for p in pf:
            if is_square((i - p) // 2):
                break
        else:
            return i


A = min_counterexample()

assert A == 5777

## [Problem 47 - Distinct primes factors](https://projecteuler.net/problem=47)

In [86]:
P = list(takewhile(lambda u: u < 10 ** 6, primes()))

def unique_factors(n):
    pf = 0
    for p in P:
        if n <= 1:
            return pf
        if n % p == 0:
            pf += 1
            while n % p == 0:
                n /= p
    return pf


def find_consecutive(l):
    c = 0
    for n in count(2):
        if unique_factors(n) == l:
            c += 1
        else:
            c = 0
        if c == l:
            return n - l + 1


A = find_consecutive(4)

assert A == 134043

## [Problem 48 - Self powers](https://projecteuler.net/problem=48)

In [87]:
N = 10 ** 10
M = range(1, 1001)
A = sum(pow(i, i, mod=N) for i in M) % N

assert A == 9110846700

## [Problem 49 - Prime permutations](https://projecteuler.net/problem=49)

In [88]:
P = set(takewhile(lambda u: u < 10000, primes()))

def gen_seq():
    for i in range(1000, 10000):
        for j in range(1, (10000 - i) // 2):
            yield i, i + j, i + 2 * j

def is_prime_perm(u):
    return (
        all(n in P for n in u) 
        and all(Counter(digits(u[0])) == Counter(digits(n)) for n in u)
    )

A = set(filter(is_prime_perm, gen_seq()))

assert A == {(1487, 4817, 8147), (2969, 6299, 9629)}

## [Problem 50 - Consecutive prime sum](https://projecteuler.net/problem=50)

In [89]:
P = list(takewhile(lambda u: u < 10 ** 6, primes()))
S = set(P)

A, n = 0, 0
for i in range(len(P)):
    s = P[i]
    for j in range(i+1, len(P)):
        s += P[j]
        if s > 10 ** 6:
            break
        if s in S and n < j - i:
            A, n = s, j - i

assert A == 997651